### 將分散的 data 轉成固定長度的 sentence 及該  sentence 的標記
1.  input : nested list of data_list 及 data_ariticle_id_list
2.  output : 句子, labels, 每個句子對應文章及在該文章的idx

output : 
* [['愛乾', '愛乾'], ['好愛', '乾']]
* [[['O', 'I'], ['O', 'I']], [['O', 'O'], ['I']]]

In [2]:
sample_txt = [[('愛','O'),('乾','I'),('愛','O'),('乾','I')],[('好', 'O'),('愛','O'),('乾','I')]]
sample_art_id = [1,2]

# Dataset

input : (文章數, )

## 需要以下code處理
1. 將各文章中的stc轉成全部轉成單一list
2. 將各文章的stc對應label也轉成list [[labels], [labels]]

### 建立 tag to id 字典
* input: 從input_format function output的 labels

### 將raw label 轉成ids

### bert dataset class建立

---

## 模型建立

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import re

In [4]:
from transformers import BertModel, BertTokenizer
import jdc

In [5]:
pretrained_model_name = 'bert-base-chinese'
tokenizer = BertTokenizer.from_pretrained(pretrained_model_name)

### helper function 

In [6]:
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()

# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

def log_sum_exp_batch(log_Tensor, axis=-1): # shape (batch_size,n,m)
    return torch.max(log_Tensor, axis)[0] + \
        torch.log(torch.exp(log_Tensor-torch.max(log_Tensor, axis)[0].view(log_Tensor.shape[0],-1,1)).sum(axis))

----

## 測試
問題: 句子的label list長度可能不是max_stc_len (再最後截斷)

In [7]:
from model import Bert_BiLstm_Crf
from dataset import bert_stc_dataset
from txt_preprocess import preprocess


file_path = 'data/train_input.data'

with open(file_path, 'r', encoding='utf-8') as f:
	data=f.readlines()#.encode('utf-8').decode('utf-8-sig')

preprocessor = preprocess(data)

stcs, labels = preprocessor.get_scts_labels(max_stc_len= 100)
tag2id_dict = preprocessor.tag2id(labels)
n_tags = len(tag2id_dict)
print('tags數: {}'.format(n_tags))
stc_labels_ids = preprocessor.label_to_ids(tag2id_dict, labels)

sentences總數: 2585
labels總數: 2585
tags數: 25


In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [11]:
# model = Bert_BiLstm_Crf(n_tags= n_tags, pretrained_model_name= pretrained_model_name, tokenizer= tokenizer).to(device)

In [8]:
batchsize = 10
maxlen = 200

In [9]:
def collate_fn(batch):
    batch.sort(key= lambda x: len(x['label']), reverse= True)
    input_ids = [batch[i]['input_ids'] for i in range(len(batch))]
    attention_masks= [batch[i]['attention_mask'] for i in range(len(batch))]
    labels = [batch[i]['label'] for i in range(len(batch))]
#     print(len(batch))
    pad_labels = []
    lens= []
    max_len= len(labels[0])
    for i in range(len(labels)):
        temp_label = [0]*max_len
        temp_label[:len(labels[i])] = labels[i]
        pad_labels.append(temp_label)
        lens.append(len(labels[i]))
    return input_ids, attention_masks, pad_labels

In [10]:
train_dataset = bert_stc_dataset(stcs, 
                                 stc_labels_ids, 
                                 tokenizer, 
                                 maxlen)
train_dataloader = DataLoader(train_dataset, 
                              batch_size= batchsize,
                             shuffle= True)

In [ ]:
bmodel = BertModel.from_pretrained(pretrained_model_name).to(device)

In [16]:
for idx, dict_ in enumerate(train_dataloader):
    input_ids = dict_['input_ids']
    attention_mask = dict_['attention_mask']
    _, out = bmodel(input_ids, attention_mask)
    print(out)
    break

ValueError: You have to specify either input_ids or inputs_embeds

In [13]:
batch = [ train_dataset[i] for i in range(10)]

In [14]:
batch.sort(key= lambda x: len(x['label']), reverse= True)
input_ids = [batch[i]['input_ids'] for i in range(len(batch))]
attention_masks= [batch[i]['attention_mask'] for i in range(len(batch))]
labels = [batch[i]['label'] for i in range(len(batch))]
#     print(len(batch))
pad_labels = []
lens= []
max_len= len(labels[0])
for i in range(len(labels)):
    temp_label = [0]*max_len
    temp_label[:len(labels[i])] = labels[i]
    pad_labels.append(temp_label)
    lens.append(len(labels[i]))

In [15]:
batch['input_ids']

TypeError: list indices must be integers or slices, not str